In [1]:
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

In [2]:
def im_multi(path):
    #Input: file path of image
    #Output: [path, {'size': size of image at path}]
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

In [3]:
def im_stats(im_stats_df): 
    #Input: dataframe of training images (type, image, path)
    #Output: dataframe of training images (type, image, path, size)
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    #p.map(f, [x, y, z]) returns a list [f[x], f[y], f[z]]
    #For all paths in the inputted dataframe im_stats_df, they are passed through im_multi
    #ret holds [[path, {'size': size of image at path}], ...]
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in 
                                                                     im_stats_d[x]['size']))
    #Adds additional column to original dataframe and formats size as 3264 4160
    return im_stats_df

In [4]:
def get_im_cv2(path):
    #Input: file path of image
    #Output: [original path, resized image]
    img = cv2.imread(path)
    resized = cv2.resize(img, (64, 64), cv2.INTER_LINEAR) #INTER_LINEAR is algorithm 
    #to downsize image
    #I could try using INTER_AREA as, according to the URL below, could be better
    #http://tanbakuchi.com/posts/comparison-of-openv-interpolation-algorithms/
    return [path, resized]

In [5]:
def normalize_image_features(paths):
    #Input: list of paths
    #Output: list of resized images that have been transposed for Conv2d layer
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_im_cv2, paths)
    #ret holds a list: [[image path, resized image], ...]
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
        #imf_d[image path in ret] = resized image
    ret = []
    fdata = [imf_d[f] for f in paths]
    #fdata holds a list: [resized image, ...]
    fdata = np.array(fdata, dtype=np.uint8)
    #fdata is now a numbpy array of ints
    fdata = fdata.transpose((0, 3, 1, 2))
    #Usually its (2, 0, 1) since it changes the image from (0, 1, 2)->(width, height, channel)
    #to (2, 0, 1)->(channel, width, height) for the Conv2d layer, 
    #but since we have 4 dimensions, it gets bumped up one to (3, 1, 2);
    #I don't get why it has 4 dimensions tho (https://skymind.ai/wiki/convolutional-network)
    fdata = fdata.astype('float32')
    fdata = fdata / 255
    #fdata now has values between 0 and 1
    return fdata

In [8]:
train = glob.glob("/Users/keerat/dev/AOSResearch/resources/Train/**/*.jpg")
#train is an array holding all of the path files in the training set
print("Number of files in training set: %", len(train))
train = pd.DataFrame([[p.split('/')[7],p.split('/')[8],p] for p in train], columns = 
                     ['type','image','path'])[::1]
#train is a dataframe holding the type (ex. "Type_1"), image name (ex. "0.jpg"), and file path
#(ex. "/Users/keerat/dev/AOSResearch/resources/Train/Type_1/0.jpg")
train = im_stats(train)
#train now has additional column with size (ex. 3264 4160)
train = train[train['size'] != '0 0'].reset_index(drop=True) #corrupt images removed
#train now has an additional column with the index reset to 0, 1, 2, 3... instead of 0, 5, 10..
print("loading train data")
train_data = normalize_image_features(train['path'])
#train_data holds a usable set of training images for the CNN
print("train data loaded")
np.save('train.npy', train_data, allow_pickle=True, fix_imports=True)
#train.npy is a file that has all of the image arrays in train_data

le = LabelEncoder()
train_target = le.fit_transform(train['type'].values) 
#train_target holds type of each image in train
#Type_1 = 0, Type_2 = 1, Type_3 = 2
#For example, if the 40th image in train is Type_1, then train_target[40] = 0
print(le.classes_)  
np.save('train_target.npy', train_target, allow_pickle=True, fix_imports=True)
#train_target.npy is a file that has all values of train_target

test = glob.glob("/Users/keerat/dev/AOSResearch/resources/test/*.jpg")
#test is an array holding all of the path files in the test set
test = pd.DataFrame([[p.split('/')[7],p] for p in test], columns = ['image','path']) [::1]
#test is a dataframe holding the image name (ex. "0.jpg"), and file path
#(ex. "/Users/keerat/dev/AOSResearch/resources/test/0.jpg")
print("loading test data")
test_data = normalize_image_features(test['path'])
#test_data holds a usable set of test images for the CNN
np.save('test.npy', test_data, allow_pickle=True, fix_imports=True)
#test.npy is a file that has all of the image arrays in test_data
print("test data loaded")
test_id = test.image.values
np.save('test_id.npy', test_id, allow_pickle=True, fix_imports=True)
#test_id.npy is a file that has all of the image names (ex. '0.jpg) in test_data

Number of files in training set: % 1478
loading train data
train data loaded
['Type_1' 'Type_2' 'Type_3']
loading test data
test data loaded


In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras import backend as K
K.set_image_dim_ordering('th')
K.set_floatx('float32')
np.random.seed(17)

/Users/keerat/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
train_data = np.load('train.npy')
train_target = np.load('train_target.npy')

In [11]:
def create_model(opt_='adamax'):
    model = Sequential()
    model.add(Convolution2D(4, 3, 3, activation='relu', dim_ordering='th', 
                            input_shape=(3, 64, 64))) 
    #Could try different input shape
    
    #Activation='relu' to discover nonlinear patterns of data
    #dim_ordering = 'th' to match (0, 3, 1, 2) images were transposed to
    #Four 3x3 filters
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    model.add(Convolution2D(8, 3, 3, activation='relu', dim_ordering='th'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    model.add(Dropout(0.2))
    #Sets a fraction of rate of input units to 0 to prevent overfitting
    model.add(Flatten())
    #Creates 1D feature vector for Dense layers
    model.add(Dense(12, activation='tanh')) #Classifies
    model.add(Dropout(0.1))
    model.add(Dense(3, activation='softmax')) #Classifies

    model.compile(optimizer=opt_, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    #Compiles all layers of model with optimizer, loss function, and metrics (to evaluate 
    #performance)
    return model

In [12]:
def cleanImages():
    datagen = ImageDataGenerator(rotation_range=0.3, zoom_range=0.3)
    #rotation_range = random rotation of images up to 0.3 degrees
    #zoom_range = random zoom of images up to a scale of 0.3
    datagen.fit(train_data)
    return datagen

In [13]:
def fitAndPredict(): #Runs data through model
    print("cleaning images")
    datagen=cleanImages() #datagen now points to the parameters in cleanImages()
    print("images cleaned")
    
    model = create_model() #model holds CNN model
    x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,
                                                               test_size=0.4, random_state=17)
    #x_train = training set of images (60% of original training set)
    #x_val_train = validation set of images (40 % of original training set)
    #y_train = types for training images
    #y_val_train = types for validation images
    
    #Training set is used to fit the parameters using back prop
    #Validation set is used to fine tune parameters to create a final model
    #Test set is used to assess model's performance
    print("fitting data")
    model.fit_generator(datagen.flow(x_train,y_train, batch_size=15, shuffle=True), 
                        nb_epoch=200, samples_per_epoch=len(x_train), 
                        verbose=2, validation_data=(x_val_train, y_val_train))
    #Training set is augmented real-time with datagen.flow
    #CNN processes images not one at a time, but in batches.  With batch_size = 15, one batch 
    #is 15 x 3 x 64 x 64.  The batch can't be too big, or else the machine can't handle it,
    #and it can't be too small or else there will be no variance within the batch.
    print("data fitted in model")
    loss, accuracy = model.evaluate(train_data, train_target)
    print('loss: ', loss, '\naccuracy: ', accuracy)
    test_data = np.load('test.npy')
    test_id = np.load('test_id.npy')
    print("creating predictions")
    predictions = model.predict_proba(test_data)
    #Runs test_data through model and returns probablibity of it being each type
    print("predictions made")
    return predictions

In [14]:
def test(isTrue): #Runs CNN on test images
    pred=fitAndPredict()
    print("creating test file")
    df = pd.DataFrame(pred, columns=['Type_1','Type_2','Type_3']) #Instantiates dataframe
    df['image_name'] = test_id #image_name holds the .jpg file name
    if (isTrue): #if(True), it will create a .csv file with the dataframe
        df.to_csv('test.csv', index=False)
        print("Test file created in users/keerat/...")
    else: #if(False), it will just show the dataframe
        print(df.to_string())

In [15]:
if __name__ == '__main__':
    
    test(isTrue = False)

cleaning images
images cleaned


/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), activation="relu", input_shape=(3, 64, 64..., data_format="channels_first")`
  after removing the cwd from sys.path.
/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_first")`
  # Remove the CWD from sys.path while we load stuff.
/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", data_format="channels_first")`
  # This is added back by InteractiveShellApp.init_path()
/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), d

fitting data


/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=2, validation_data=(array([[[..., steps_per_epoch=59, epochs=200)`


Epoch 1/200
 - 10s - loss: 1.0011 - acc: 0.5377 - val_loss: 1.0484 - val_acc: 0.5203
Epoch 2/200
 - 9s - loss: 1.0103 - acc: 0.5199 - val_loss: 1.0376 - val_acc: 0.5203
Epoch 3/200
 - 8s - loss: 0.9756 - acc: 0.5403 - val_loss: 1.0226 - val_acc: 0.5203
Epoch 4/200
 - 8s - loss: 0.9860 - acc: 0.5154 - val_loss: 1.0380 - val_acc: 0.5186
Epoch 5/200
 - 7s - loss: 0.9759 - acc: 0.5513 - val_loss: 1.0192 - val_acc: 0.5186
Epoch 6/200
 - 7s - loss: 0.9773 - acc: 0.5356 - val_loss: 1.0293 - val_acc: 0.5203
Epoch 7/200
 - 7s - loss: 0.9596 - acc: 0.5490 - val_loss: 1.0215 - val_acc: 0.5203
Epoch 8/200
 - 7s - loss: 0.9538 - acc: 0.5346 - val_loss: 1.0102 - val_acc: 0.4966
Epoch 9/200
 - 6s - loss: 0.9534 - acc: 0.5314 - val_loss: 1.0048 - val_acc: 0.5084
Epoch 10/200
 - 7s - loss: 0.9499 - acc: 0.5267 - val_loss: 1.0015 - val_acc: 0.5051
Epoch 11/200
 - 6s - loss: 0.9186 - acc: 0.5603 - val_loss: 0.9754 - val_acc: 0.4983
Epoch 12/200
 - 7s - loss: 0.9086 - acc: 0.5931 - val_loss: 0.9809 - val_

Epoch 98/200
 - 7s - loss: 0.7459 - acc: 0.6700 - val_loss: 1.0043 - val_acc: 0.5338
Epoch 99/200
 - 7s - loss: 0.7319 - acc: 0.6835 - val_loss: 1.0038 - val_acc: 0.5236
Epoch 100/200
 - 8s - loss: 0.7348 - acc: 0.6893 - val_loss: 1.0072 - val_acc: 0.5084
Epoch 101/200
 - 8s - loss: 0.7269 - acc: 0.6735 - val_loss: 1.0010 - val_acc: 0.5118
Epoch 102/200
 - 7s - loss: 0.7297 - acc: 0.6825 - val_loss: 1.0259 - val_acc: 0.5236
Epoch 103/200
 - 7s - loss: 0.7806 - acc: 0.6612 - val_loss: 1.0041 - val_acc: 0.5152
Epoch 104/200
 - 7s - loss: 0.7298 - acc: 0.6677 - val_loss: 1.0051 - val_acc: 0.5321
Epoch 105/200
 - 8s - loss: 0.6897 - acc: 0.7174 - val_loss: 1.0195 - val_acc: 0.5355
Epoch 106/200
 - 9s - loss: 0.7365 - acc: 0.6655 - val_loss: 1.0160 - val_acc: 0.5270
Epoch 107/200
 - 11s - loss: 0.7107 - acc: 0.6948 - val_loss: 1.0334 - val_acc: 0.5321
Epoch 108/200
 - 7s - loss: 0.7214 - acc: 0.6903 - val_loss: 1.0062 - val_acc: 0.5372
Epoch 109/200
 - 8s - loss: 0.7457 - acc: 0.6835 - val_

 - 8s - loss: 0.6063 - acc: 0.7423 - val_loss: 1.0980 - val_acc: 0.5169
Epoch 194/200
 - 10s - loss: 0.6378 - acc: 0.7030 - val_loss: 1.1254 - val_acc: 0.5355
Epoch 195/200
 - 11s - loss: 0.6439 - acc: 0.7321 - val_loss: 1.1095 - val_acc: 0.5287
Epoch 196/200
 - 9s - loss: 0.6115 - acc: 0.7367 - val_loss: 1.0966 - val_acc: 0.5321
Epoch 197/200
 - 8s - loss: 0.6581 - acc: 0.6953 - val_loss: 1.1022 - val_acc: 0.5304
Epoch 198/200
 - 8s - loss: 0.6492 - acc: 0.7220 - val_loss: 1.1107 - val_acc: 0.5169
Epoch 199/200
 - 8s - loss: 0.6614 - acc: 0.7030 - val_loss: 1.0952 - val_acc: 0.5304
Epoch 200/200
 - 8s - loss: 0.6082 - acc: 0.7414 - val_loss: 1.0923 - val_acc: 0.5186
data fitted in model
1478/1478 [==============================] - 7s 5ms/step
loss:  0.7218181119074841 
accuracy:  0.7083897157515499
creating predictions
predictions made
creating test file
       Type_1    Type_2    Type_3 image_name
0    0.070165  0.431213  0.498623      0.jpg
1    0.013187  0.028032  0.958781      1.j